### W2V Modeling with GloVe Vectors

This notebook does w2v modeling on the dataset using the pre-trained GloVe vectors as a base. This method is the one used to generate the w2v models found on the [Philosophy Data Project website](http://www.philosophydata.com).

First we import some libraries and get the data.

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser


# a function for testing w2v pairs
def test_w2v(model, pairs):
  for (pos, neg) in pairs:
    math_result = model.most_similar(positive=pos, negative=neg)
    print(f'Positive - {pos}\tNegative - {neg}')
    [print(f"- {result[0]} ({round(result[1],5)})") for result in math_result[:5]]
    print()
    



In [ ]:
df = pd.read_csv('../input/history-of-philosophy/phil_nlp.csv')

df.sample(5)

In [ ]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

# check how it worked
print(df.iloc[290646]['sentence'])
df['gensim_tokenized'][290646]

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [ ]:
# load the vectors. other vector sizes were used but yielded generally less sensible models
glove_file = datapath('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
#defining some pairs to test out
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [ ]:
# check out how GloVe works on our test pairs
test_w2v(glove_vectors, pairs_to_try)

Now we want these to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

In [ ]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [ ]:
test_w2v(base_model_wv, pairs_to_try)

These seem to reflect the true uses of words in German Idealism. 'Self' + 'consciousness' is rightly associated with 'self consciousness' and 'moral' + 'law' with 'moral law'. It even identifies the German Idealist tendency to unify logic and method. 

These vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

In [ ]:
# turn the process into a function for easy use in the future
def train_glove(source_type, source, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df[source_type] == source]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [ ]:
ph_model = train_glove(source_type='school', source='phenomenology', glove_vectors=glove_vectors)
test_w2v(ph_model, pairs_to_try)

Using the phenomenology vectors on some central terms of German idealism once again yields some pretty compelling results, except for where the words are rarely used by the phenomenologists. This is to be expected. Let's try the word vectors on some central terms of phenomenology.

In [ ]:
pairs_to_try = [(['perception'], []),
                (['dasein'], []),
                (['consciousness'], []),
                (['method'], []),]

test_w2v(ph_model, pairs_to_try)

These look pretty strong. Overall, the GloVe-trained vectors seem to be an effective tool for revealing how a word is used by a school. 

### Training on every School & Author

To further explore this, we'll train w2v models in this way for each school and examine how each of them looks at the same word - 'philosophy.' We can use these in our future dashboard work.

In [ ]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove(source_type='school', source=school, glove_vectors=glove_vectors)
  print(f'{school} completed')

In [ ]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. Continental thinkers mentioning unreason, analytic philosophers focusing on semantics, and phenomenologists associating philosophy with a method all track well. The ones that don't make sense are those that don't problematize the nature of philosophy to any great degree - capitalist thinkers aren't out there trying to discuss the nature of philosophy.

We'd also like vectors trained for each individual author.

In [ ]:
for author in df['author'].unique():
  w2v_dict[author] = train_glove('author', author, glove_vectors=glove_vectors)
  print(f'{author} completed')

### Training on the Full Corpus

In [ ]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv

As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [ ]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])

This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

### Exporting

In [ ]:
all_text_wv.save_word2vec_format('/w2v_models/w2v_for_nn.bin')
all_text_wv.save('/w2v_models/w2v_for_nn.wordvectors')

for source in w2v_dict.keys():
  w2v_dict[source].save(f'/w2v_models/{source}_w2v.wordvectors')